In [9]:
import numpy as np 
import pandas as pd 

MODEL_NAME='multi_bibasic_lstm'
HPARAMS_STR='lr5.00e-04,hs128x64'
MODEL_V= '01'
INPUT_V='00'
INPUT_DIR= '../datasets/csvs/'
OUTPUT_DIR='../outputs/'

INPUT_PATH = '{:}{:}'.format(INPUT_DIR, 'zhou_valid.csv')
OUTPUT_PATH= '{:}{:}/{:}/{:}/{:}'.format(OUTPUT_DIR,MODEL_NAME,HPARAMS_STR,MODEL_V, 'Yhat_valid.csv')
TARGET_PATH= '{:}{:}/{:}/{:}/{:}'.format(OUTPUT_DIR,MODEL_NAME,HPARAMS_STR,MODEL_V, 'evaluation_valid.csv')



print(INPUT_PATH)
print(OUTPUT_PATH)

../datasets/csvs/zhou_valid.csv
../outputs/multi_bibasic_lstm/lr5.00e-04,hs128x64/01/Yhat_valid.csv


In [10]:
df_input= pd.read_csv(INPUT_PATH, index_col=0)
df_output=pd.read_csv(OUTPUT_PATH, index_col=0)
print(df_input.shape, df_output.shape)

(12298, 10) (12298, 1)


In [11]:
df= df_input.join(df_output, how='left')
print(df.shape)

(12298, 11)


In [12]:
df.to_csv(TARGET_PATH)

In [14]:
df.head()

,ID,S,P,P_S,FORM,LEMMA,PRED,M_R,ARG,ARG_Y,Yhat
IDX,,,,,,,,,,,
123846,1,2879,1,5100,Entre,entre,estar,0,(A2*,A2,a2
123847,2,2879,1,5100,os,o,estar,0,*,A2,a2
123848,3,2879,1,5100,convidados,convidado,estar,0,*),A2,a2
123849,4,2879,1,5100,estão,estar,estar,1,(V*),V,v
123850,5,2879,1,5100,o,o,estar,1,(A1*,A1,a2
